In [ ]:
import os
import random

import numpy as np 
from numpy import load
from numpy import savez_compressed
import nibabel as nib
import tensorflow as tf

from nets.unet3d import *
from utils_tumores import *

np.set_printoptions(precision=2, suppress=True)
PARENT_DIR = os.getcwd()
MODEL_PATH = os.path.join(PARENT_DIR, "models", "3D")

In [ ]:
print(output_x)

In [ ]:
print(tf.__version__)
tf.config.list_physical_devices("GPU") 

In [ ]:
# Crea los directorios si no existen, el unico requisito es que se tenga la DB en una carpeta llamada NFBS_Dataset 
# Dentro del mismo directorio que este codigo
volumes_folders = count_volumes(dataset_folder)
paths, train_files, val_files, test_files = make_dirs(volumes_folders)
paths.keys()


# Data split

In [ ]:
print(len(train_files))
print(len(val_files))
print(len(test_files))

In [ ]:
print(train_files[0])


In [ ]:
paths["SUBVOLUME_FOLDER"]

In [ ]:
os.path.join(paths["SUBVOLUME_FOLDER"],"BraTS20_Training_344")

In [ ]:
# Dividimos la lista de las carpetas de pacientes en train, val y test
''' train_files, val_files = train_test_split(paths["SAMPLES"], test_size=0.2, random_state=42)
test_files, val_files = train_test_split(val_files, test_size=0.5, random_state=42) '''

train_images=[]
train_images_mask=[]

for sample in train_files:
    for subvol in sorted(os.listdir(os.path.join(paths["SUBVOLUME_FOLDER"],sample))):
        train_images.append(os.path.join(paths["SUBVOLUME_FOLDER"],sample,subvol))
        

    for subvol in sorted(os.listdir(os.path.join(paths["SUBVOLUME_MASK_FOLDER"],sample))):
        train_images_mask.append(os.path.join(paths["SUBVOLUME_MASK_FOLDER"],sample,subvol))

val_images=[]
val_images_mask=[]
for sample in val_files:
    for subvol in sorted(os.listdir(os.path.join(paths["SUBVOLUME_FOLDER"],sample))):
        val_images.append(os.path.join(paths["SUBVOLUME_FOLDER"],sample,subvol))
        

    for subvol in sorted(os.listdir(os.path.join(paths["SUBVOLUME_MASK_FOLDER"],sample))):
        val_images_mask.append(os.path.join(paths["SUBVOLUME_MASK_FOLDER"],sample,subvol))


test_images=[]
test_images_mask=[]
# Agregamos una lista mas para crear la ruta de almacenamiendo de las predicciones
test_predict_images=[]

for sample in test_files:
    for subvol in sorted(os.listdir(os.path.join(paths["SUBVOLUME_FOLDER"],sample))):
        test_images.append(os.path.join(paths["SUBVOLUME_FOLDER"],sample,subvol))
        test_predict_images.append(os.path.join(paths["RESULTS_3D"],sample,subvol))
        

    for subvol in sorted(os.listdir(os.path.join(paths["SUBVOLUME_MASK_FOLDER"],sample))):
        test_images_mask.append(os.path.join(paths["SUBVOLUME_MASK_FOLDER"],sample,subvol))


In [ ]:
paths["RESULTS_3D"]

In [ ]:
print(len(train_images))
print(len(train_images_mask))

# Dataloader

In [ ]:
def load_image(file, label):
    dict_data_file = load(file.numpy().decode('utf-8'))
    dict_data_label = load(label.numpy().decode('utf-8'))
    # extract the first array
    nifti = dict_data_file['arr_0']
    label = dict_data_label['arr_0']
    #nifti = np.asarray(nib.load(file.numpy().decode('utf-8')).get_fdata()).astype(np.int16)
    #label = np.asarray(nib.load(label.numpy().decode('utf-8')).get_fdata()).astype(np.int16)
    return nifti, label


@tf.autograph.experimental.do_not_convert
def load_image_wrapper(file, label):
    image, label = tf.py_function(load_image, [file, label], [tf.float32, tf.float32])
    image.set_shape(tf.TensorShape([output_x, output_y, output_z]))
    label.set_shape(tf.TensorShape([output_x, output_y, output_z]))
    return image, label


train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_images_mask))
train_dataset = train_dataset.shuffle(3, reshuffle_each_iteration=False)
train_dataset = train_dataset.map(load_image_wrapper, num_parallel_calls=32)
train_dataset = train_dataset.batch(8, drop_remainder=False)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_images_mask))
val_dataset = val_dataset.shuffle(3, reshuffle_each_iteration=False)
val_dataset = val_dataset.map(load_image_wrapper, num_parallel_calls=32)
val_dataset = val_dataset.batch(8, drop_remainder=False)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_images_mask))
test_dataset = test_dataset.map(load_image_wrapper, num_parallel_calls=32)
test_dataset = test_dataset.batch(8, drop_remainder=False)

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
model_unet=unet_3D(output_x, output_y, output_z)
model_unet.compile(optimizer=opt, loss = 'binary_crossentropy', 
                    metrics = ['accuracy', dice_coeff]) 

In [ ]:
callbacks = ModelCheckpoint(MODEL_PATH, 
                            monitor='val_dice_coeff',
                            verbose=1, 
                            mode='max',
                            save_best_only=True)

In [ ]:
history = model_unet.fit(train_dataset,
                         validation_data=val_dataset,
                         epochs=3,
                         callbacks=[callbacks, tf.keras.callbacks.EarlyStopping(verbose=2, patience=2,min_delta=0.001, monitor='val_dice_coeff')]) #Guardar la mejor epoca para validación

In [ ]:
results = model_unet.evaluate(test_dataset)


# Predicciones

In [ ]:
model_unet_predict=unet_3D(output_x, output_y, output_z)

In [ ]:
model_unet_predict.load_weights(MODEL_PATH)

In [ ]:
for sub_volume in range(len(test_images)):
    #Cargamos cada imagen como un array
    dict_data = load(test_images[sub_volume])
    # extract the first array
    nifti = dict_data['arr_0']
    #Convertimos a tensor
    image = tf.convert_to_tensor(nifti, dtype=tf.float32)
    #Agregamos la dimension del batch y no recuerdo cual es la otra
    image = tf.expand_dims(image, axis=3)
    image = tf.expand_dims(image, axis=0)
    #Se realiza la prediccion
    prediction = model_unet.predict(image)
    #Elimina la dimension del batch
    prediction = prediction.squeeze()
    #Aplicamos un threshold para el valor de cada pixel
    prediction[prediction>=0.5]=1.0
    prediction[prediction<0.5]=0.0
    #Guardamos la imagen
    savez_compressed(test_predict_images[sub_volume], prediction)
    

In [ ]:
test_files

# Reconstruccion de los volumenes de test

In [ ]:
#Esta funcion esta muy bien documentada en el modulo utils.py
pathe=reconstruction(test_files=test_files, path_resultados=paths["RESULTS_3D"],path_rotation_matrix=paths["ROTATION_MATRIX"])